Берем результаты других ноутбуков и объединяем их тут

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join, isdir

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

In [3]:
datename = 'magic'

In [4]:
path = f'./eval/total/'
dirs = [join(path, f) for f in listdir(path) if isdir(join(path, f))]

In [5]:
results = []
for d in dirs:
    if datename in d:
        res = [pd.read_csv(join(d, f), header=[0, 1], skipinitialspace=True) for f in listdir(d) if isfile(join(d, f)) and 'csv' in f]
        results.extend(res)

results = pd.concat(results)
results = results.rename(columns={'Unnamed: 0_level_0':'', 
                                  'Unnamed: 1_level_0':'', 
                                  'Unnamed: 2_level_0':''})

In [12]:
import numpy as np
import json

with open('/workspace/TabDDPM_copy/TabDDPM/Model_names_simple.json', 'r') as openfile:
    naming = json.load(openfile)

def get_positions(s, substring):
    positions = []
    start = 0
    while True:
        start = s.find(substring, start)
        if start == -1:
            break
        positions.append(start)
        start += len(substring)
    return positions
    
def get_simple_name(model_name):
    try:
        pos = get_positions(model_name, '_')[-2]
        model_name_short = model_name[pos+1:]

        if naming.get(model_name_short):
            return naming[model_name_short]
        else:
            pos = get_positions(model_name, '_')[-3]
            model_name_short = model_name[pos+1:]
            return naming[model_name_short]
    except:
        return 'TabDDPM'

results[('', 'Model name')] = results[('', 'Type')].apply(lambda x: get_simple_name(x))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = results[[
    ('', 'Model name') ,
    ('similarity', 'Column Shapes Score, %') ,
    ('similarity', 'Column Pair Trends Score, %') ,
    ('similarity', 'Overall Score (Average), %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation, %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation std, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score std, %') ,
    ('mle', 'ROC - AUC обучения на синтетических данных') ,
    ('mle', 'ROC - AUC обучения на синтетических данных, std') ,
    ('detection', 'Score') ,
    ('DCR', 'Score') ,
    ('quality', 'alpha precision') ,
    ('quality', 'beta recall') ,
]].sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

df

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
0     Cosine 1              99.148690                   99.158099   
1      Steps 3              99.254904                   98.935364   
2      Steps 2              99.274554                   98.689862   
3   Constant 4              99.176837                   98.688728   
4     Linear 1              99.261277                   98.547238   
5      TabDDPM              98.910238                   98.884369   
6     Cosine 2              99.217729                   98.478857   
7      Steps 1              99.115763                   98.545120   
8      Steps 4              99.263402                   98.367415   
9   Constant 2              99.015922                   98.500702   
10    Cosine 3              99.120543                   98.334696   
11  Constant 5              98.837481                   98.525085   
12    Linear 2              99.054690                   97.666664   
13     Steps 5              99.119481                   97.518794   
14  Constant 1              98.957504                   97.487295   
15  Constant 3              98.833764                   97.207305   
16  Constant 6              97.146013                   96.347337   

                               \
   Overall Score (Average), %   
0                   99.153394   
1                   99.095134   
2                   98.982208   
3                   98.932783   
4                   98.904258   
5                   98.897304   
6                   98.848293   
7                   98.830442   
8                   98.815408   
9                   98.758312   
10                  98.727619   
11                  98.681283   
12                  98.360677   
13                  98.319138   
14                  98.222399   
15                  98.020534   
16                  96.746675   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             0.851310   
1                                             0.745096   
2                                             0.725446   
3                                             0.823163   
4                                             0.738723   
5                                             1.089762   
6                                             0.782271   
7                                             0.884237   
8                                             0.736598   
9                                             0.984078   
10                                            0.879457   
11                                            1.162519   
12                                            0.945310   
13                                            0.880519   
14                                            1.042496   
15                                            1.166236   
16                                            2.853987   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
0                                                 0.267489   
1                                                 0.296512   
2                                                 0.320819   
3                                                 0.289313   
4                                                 0.180007   
5                                                 0.374344   
6                                                 0.263289   
7                                                 0.312559   
8                                                 0.318241   
9                                                 0.302799   
10                                                0.369259   
11                                                0.370400   
12                                                0.314719   
13                                                0.24

In [13]:
df[df[('', 'Model name')].isin(['TabDDPM', 'Constant 2', 'Constant 4', 'Cosine 3'])]

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
3   Constant 4              99.176837                   98.688728   
5      TabDDPM              98.910238                   98.884369   
9   Constant 2              99.015922                   98.500702   
10    Cosine 3              99.120543                   98.334696   

                               \
   Overall Score (Average), %   
3                   98.932783   
5                   98.897304   
9                   98.758312   
10                  98.727619   

                                                        \
   Error rate (%) of column-wise density estimation, %   
3                                             0.823163   
5                                             1.089762   
9                                             0.984078   
10                                            0.879457   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
3                                                 0.289313   
5                                                 0.374344   
9                                                 0.302799   
10                                                0.369259   

                                                            \
   Error rate (%) of pair-wise column correlation score, %   
3                                                 1.311272   
5                                                 1.115631   
9                                                 1.499298   
10                                                1.665304   

                                                                \
   Error rate (%) of pair-wise column correlation score std, %   
3                                                     2.605576   
5                                                     1.121759   
9                                                     2.577808   
10                                                    3.896525   

                                          mle  \
   ROC - AUC обучения на синтетических данных   
3                                    0.932952   
5                                    0.924149   
9                                    0.951595   
10                                   0.934294   

                                                   detection       DCR  \
   ROC - AUC обучения на синтетических данных, std     Score     Score   
3                                         0.004988  0.995042  0.538439   
5                                         0.005778  0.985271  0.512034   
9                                         0.007142  0.987209  0.555030   
10                                        0.004516  0.997994  0.520972   

           quality              
   alpha precision beta recall  
3         0.994272    0.480188  
5         0.978676    0.491915  
9         0.977805    0.631978  
10        0.993891    0.516112

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

\
              Model                              Type   Data   
0   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  magic   
1   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  magic   
2   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  magic   
3   tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  magic   
4   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  magic   
5           TabDDPM                           tabddpm  magic   
6   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  magic   
7   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  magic   
8   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  magic   
9   tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  magic   
10  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  magic   
11  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  magic   
12  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  magic   
13  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  magic   
14  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  magic   
15  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  magic   
16  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  magic   

        base_metrics                                                  \
   Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0           0.833056           0.832708      0.872042       0.852273   
1           0.833056           0.834167      0.872042       0.868273   
2           0.833056           0.830142      0.872042       0.862871   
3           0.833056           0.831461      0.872042       0.871816   
4           0.833056           0.832639      0.872042       0.858995   
5           0.833056           0.832572      0.871856       0.871200   
6           0.833056           0.831601      0.872042       0.870156   
7           0.833056           0.833125      0.872042       0.860522   
8           0.833056           0.827701      0.872042       0.860699   
9           0.833056           0.831104      0.872042       0.850433   
10          0.833056           0.831592      0.872042       0.861201   
11          0.833056           0.831822      0.872042       0.862110   
12          0.833056           0.831180      0.872042       0.877461   
13          0.833056           0.832013      0.872042       0.865600   
14          0.833056           0.829716      0.877756       0.871795   
15          0.833056           0.830189      0.872042       0.859821   
16          0.833056           0.835165      0.872042       0.843260   

                                                               similarity  \
   Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %   
0                   0.041885              2.266978              99.148690   
1                  -0.133267              0.432161              99.254904   
2                   0.349908              1.051650              99.274554   
3                   0.191560              0.025929              99.176837   
4                   0.050058              1.496096              99.261277   
5                   0.058197              0.075275              98.910238   
6                   0.174736              0.216263              99.217729   
7                  -0.008257              1.321003              99.115763   
8                   0.642836              1.300764              99.263402   
9                   0.234414              2.477987              99.015922   
10                  0.175779              1.243108              99.120543   
11                  0.148195              1.138922              98.837481   
12                  0.225293             -0.621428              99.054690   
13                  0.125218              0.738694              99.119481   
14                  0.400967              0.679077              98.957504   
15                  0.344250              1.401426              98.833764   
16                 -0.253087              3.300476              97.146013   

            

In [8]:
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False
                   ).reset_index(drop=True).to_excel(f'./_{datename}_notebooks/Result_united.xlsx')